In [8]:
from src.pispec import PiSpec
from src.trace_utils import TraceParams
import matplotlib.pyplot as plt
from src.tracecontroller import TraceController

# initialize the interface
pispec = PiSpec()
print(pispec.tracecontroller.device)
# alternate the datapoints between two detectors? 
# Thats a tracecontroller things
pispec.set_actinic(100)

/dev/ttyACM0


'a100;'

In [ ]:
for x in (0, 255, 0):
    print(pispec.set_actinic(x))
    pispec.wait(1)


In [ ]:
# # pispec.reconnect()
# power("off")
resp = pispec.power("on")
print(resp)

resp = pispec.power("off")
print(resp)

In [ ]:
for i in range(0, 255):
    print(pispec.set_actinic(i))
    pispec.wait(0.25)


In [ ]:
pispec.set_actinic(0)

In [22]:


def act_pulse(pulse_length:float, intensity: int):
    resp1 = pispec.tracecontroller.set_parameters(intensity)
    pispec.wait(pulse_length)
    resp2 = pispec.tracecontroller.set_parameters(intensity)
    # print(f"{resp1}, {resp2}")

# for i in range(0, 255):
#     act_pulse()

int_list = [x*10 for x in range(0, 25)] 
# int_list = [255, 100, 50, 100, 255]

for i in range(0, 255):
    int_list.reverse()

    for intensity in int_list:
        # print(intensity)
        pispec.set_actinic(intensity)
        pispec.wait(1)
        
    

        


KeyboardInterrupt: 

In [ ]:
pispec = PiSpec()
pispec.power("on")
#experiment variables
exp_name = "testBadger"
trace_params = [TraceParams(num_points=1000,
    pulse_interval = 1000,
    meas_led_ir = 0,
    meas_led_vis = 2,
    pulse_length = 60,
    sat_pulse_begin = 500,
    sat_pulse_end = 600,
    pulse_mode = 1,
    trigger_delay = 0,
    trace_note = "debug",
    act_intensity = [0, 6000, 0]
) for x in range(1, 7)]

# begin experiment
pispec.init_experiment(exp_name=exp_name)

for param in trace_params:
    
    # setup the trace paramaters
    pispec.setup_trace(param)

    # see if the trace parameters are in place
    # print(f"The current trace parameters are: {pispec.tracecontroller.get_parameters()}")

    # run trace
    print(f"LED: {param.meas_led_vis}")
    status = pispec.run_trace(timeout=1000)
    # print(f"pispec.run_trace completed with status: {status}")

    # wait between traces
    pispec.wait(2)

print(f"Experiment '{exp_name}' concluded, trace data is ready for processing.")


In [ ]:
# dfs = pispec.get_data()
dfs = pispec.get_data()
df = dfs[0]


In [ ]:
print(df.head())


In [ ]:
# row means of all the data points
df["val"] = df[['aq_0', 'aq_1', 'aq_2', 'aq_3', 'aq_4']].mean(numeric_only = True, axis=1)

df["V"] = df["val"] / 6535

plt.plot(df['time_us'], df['V'])
plt.ylim(00.3, 0.61)
plt.show()

In [ ]:
plt.plot(df['time_us'], df['V'])
plt.ylim(0, 10)
plt.show()

In [ ]:
(x='time_us', y='val')
plt.show()

# plt.plot(df['time_us'], df['aq_0'])
# plt.show()

In [ ]:

# pispec.tracecontroller.modify_actinic(intensity=500)
# time.sleep(5)
